In [1]:
# Code Schritt für Schritt ausführen, um alle Daten in der DB "footballmap" zu aktualisieren
# Bemerkung: Die Tabelle "land" wird nicht aktualisiert. Diese Daten bleiben wie bisher.

# Stefan Sidler, 02.04.2024, aktualisiert am 23.04.2024

In [ ]:
#!!!!! Zugangsdaten zur DB wurden aus Sicherheitsgründen entfernt
#!!!!! Pfade und Dateinamen müssen bei Bedarf angepasst werden

In [35]:
# Verbindung zur PostgreSQL-DB aufbauen

import requests
import psycopg2
from io import BytesIO
import json

# Verbindung zur PostgreSQL-Datenbank herstellen
conn = psycopg2.connect(
    dbname="footballmap",
    user="postgres",
    password="xxx",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [ ]:
# Schritt 1: Bestehende Daten löschen (Standardmässig alle Tabellen ausser "land")
# Liste der Tabellen anpassen, aus denen Daten gelöscht werden sollen
tables_to_clear = ['Stadium', 'Liga', 'Club', 'Spieler', 'Transfer', 'clubstatistik']

try:
    # Daten aus den Tabellen löschen
    for table in tables_to_clear:
        cur.execute(f"DELETE FROM {table};")
        print(f"Daten aus Tabelle {table} wurden erfolgreich gelöscht.")
    
    # Transaktion bestätigen
    conn.commit()
    
    print("Alle Daten wurden erfolgreich gelöscht, außer denen in der Tabelle 'Land'.")
    
except Exception as e:
    # Transaktion rückgängig machen und Fehler ausgeben
    conn.rollback()
    print("Fehler beim Löschen der Daten:", e)

In [6]:
# Schritt 2: Daten in Tabelle "liga" importieren

# Pfad und Dateiname bei Bedarf anpassen
with open('C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/footballmap_v0.1/AlleLigen_formatiert.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Iteriere durch die JSON-Daten und füge sie in die Datenbank ein
for entry in data:
    cur.execute("""
        INSERT INTO Liga (liga_nr, name, land)
        VALUES (%s, %s, %s)
        ON CONFLICT (liga_nr) DO NOTHING
    """, (entry['liga_nr'], entry['name'], entry['land']))

# Transaktion bestätigen
conn.commit()


In [ ]:
# Schritt 3: Daten in Tabelle "stadion" importieren

# Pfad und Dateiname bei Bedarf anpassen
with open('C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/footballmap_v0.1/AlleLigenAlleClubs_formatiert.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Iteriere durch die JSON-Daten und füge sie in die Datenbank ein
for entry in data:
    # Überprüfung auf leere Werte
    stadium_name = entry['stadium_name'] if entry['stadium_name'] else None
    kapazität = entry['kapazität'] if entry['kapazität'] else None
    lon = entry['lon'] if entry['lon'] else None
    lat = entry['lat'] if entry['lat'] else None

    cur.execute("""
        INSERT INTO Stadium (stadium_nr, stadium_name, kapazität, lon, lat)
        VALUES (%s, %s, %s, %s, %s)
    """, (entry['stadium_nr'], stadium_name, kapazität, lon, lat))

# Transaktion bestätigen
conn.commit()

In [25]:
# Schritt 4: Daten in Tabelle "club" importieren

# Pfad und Dateiname bei Bedarf anpassen
with open('C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/footballmap_v0.1/AlleLigenAlleClubs_formatiert.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

try:
    # Iteriere durch die JSON-Daten und füge sie in die Datenbank ein
    for entry in data:
        cur.execute("""
            INSERT INTO Club (club_nr, name, official_name, logo_link, ort, land, stadium_nr, liga_nr)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (club_nr) DO NOTHING
        """, (
            entry['club_nr'],
            entry['name'],
            entry.get('officialName', None),
            entry.get('logo', None),
            entry['ort'],
            entry['land'],
            entry['stadium_nr'],
            entry['liga_id_tm']
        ))

    # Transaktion bestätigen
    conn.commit()

except Exception as e:
    print(f"Fehler beim Einfügen der Daten: {e}")
    conn.rollback()  # Rollback der Transaktion im Fehlerfall


In [ ]:
# Schritt 5: Daten in Tabelle "clubstatistik" importieren
# bei der definitiven Version (v4) ist diese Tabelle nicht mehr in der DB implementiert

# Pfad und Dateiname bei Bedarf anpassen
with open('C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/footballmap_v0.1/teams_data2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Iteriere durch die JSON-Daten und füge sie in die Datenbank ein
for entry in data:
    cur.execute("""
        INSERT INTO clubstatistik (club_nr, rang, spiele, gewonnen, unentschieden, verloren, tore, gegentore, punkte)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        entry['club_nr'],
        entry['rang'],
        entry['spiele'],
        entry['gewonnen'],
        entry['unentschieden'],
        entry['verloren'],
        entry['tore'],
        entry['gegentore'],
        entry['punkte']
    ))


# Transaktion bestätigen
conn.commit()



In [ ]:
# Schritt 6 (Teil 1 von 2): Daten in Tabelle "spieler" importieren

import json
import psycopg2
from psycopg2 import sql
import logging

# Logging konfigurieren
logging.basicConfig(level=logging.INFO)

# Datenbankverbindung
try:
    conn = psycopg2.connect(
        dbname="footballmap",
        user="postgres",
        password="xxx",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    # Pfad zur JSON-Datei
    json_file_path = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/footballmap_v0.1/alle_spieler_daten.json"

    # Funktion zum Umwandeln von Datumsangaben in SQL-Datumsformat
    def convert_date(date_str):
        months = {
            "Jan": "01",
            "Feb": "02",
            "Mar": "03",
            "Apr": "04",
            "May": "05",
            "Jun": "06",
            "Jul": "07",
            "Aug": "08",
            "Sep": "09",
            "Oct": "10",
            "Nov": "11",
            "Dec": "12"
        }
        
        # Datum im Format "Mon DD, YYYY" aufsplitten
        parts = date_str.split()
        
        # Monat abrufen und in das richtige Format umwandeln
        month = months.get(parts[0], '01')  # Standardwert für den Monat: Januar
        
        # Tag und Jahr extrahieren
        day = parts[1].strip(',')
        year = parts[2]
        
        # Datum im SQL-Datumsformat zurückgeben
        return f"{year}-{month}-{day}"

    # Öffnen und Analysieren der JSON-Datei im UTF-8-Format
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Iterieren durch die Daten und sie in die Datenbank einfügen oder aktualisieren
    for player_data in data:
        try:
            spieler_nr = int(player_data.get('spieler_nr', -1))  # Konvertiere zu INTEGER
            name = player_data.get('name', 'Unknown')[:100]  # Begrenze auf 100 Zeichen
            geburtsdatum = convert_date(player_data.get('geburtsdatum', 'Jan 1, 1970'))
            nationalität = player_data.get('nationalität', 'Unknown')[:100]  # Begrenze auf 100 Zeichen
            körpergrösse = float(player_data.get('körpergrösse', '0.00').replace("m", "").replace(",", ".")) * 100 if player_data.get('körpergrösse') else None
            marktwert_aktuell = float(player_data.get('marktwert', '0').replace("€", "").replace("k", "000").replace("m", "000000"))
            position = player_data.get('position', 'Unknown')[:50]  # Begrenze auf 50 Zeichen
            starker_fuss = player_data.get('starker_fuss', None) if player_data.get('starker_fuss', None) != "Information nicht gefunden" else None
            vereinsbeitritt = convert_date(player_data.get('vereinsbeitritt', 'Jan 1, 1970'))
            club_nr = int(player_data.get('vereinsnummer', None)) if player_data.get('vereinsnummer', None) != "Vereinsnummer nicht gefunden" else None
            
            # SQL-INSERT- oder UPDATE-Anweisung vorbereiten und Spielerdaten einfügen oder aktualisieren
            insert_update_query = sql.SQL("""
            INSERT INTO Spieler (spieler_nr, name, geburtsdatum, nationalität, körpergrösse, marktwert_aktuell, position, starker_fuss, vereinsbeitritt, club_nr)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (spieler_nr) 
            DO UPDATE SET 
                trikotnummer = COALESCE(EXCLUDED.trikotnummer, Spieler.trikotnummer),
                geburtsort = COALESCE(EXCLUDED.geburtsort, Spieler.geburtsort),
                geburtsland = COALESCE(EXCLUDED.geburtsland, Spieler.geburtsland),
                starker_fuss = EXCLUDED.starker_fuss,
                nationalität = COALESCE(EXCLUDED.nationalität, Spieler.nationalität);
            """)
            
            # Wenn club_nr nicht None ist, füge sie dem data_tuple hinzu
            if club_nr is not None:
                data_tuple = (spieler_nr, name, geburtsdatum, nationalität, körpergrösse, marktwert_aktuell, position, starker_fuss, vereinsbeitritt, club_nr)
            else:
                data_tuple = (spieler_nr, name, geburtsdatum, nationalität, körpergrösse, marktwert_aktuell, position, starker_fuss, vereinsbeitritt)
            
            cur.execute(insert_update_query, data_tuple)

        except Exception as e:
            logging.error(f"Fehler beim Verarbeiten des Spielers {name}. Fehler: {e}")

    # Transaktion bestätigen
    conn.commit()
    logging.info("Daten wurden erfolgreich in die Datenbank eingefügt bzw. aktualisiert!")

except Exception as e:
    logging.error(f"Fehler beim Einfügen oder Aktualisieren der Daten: {e}")
    conn.rollback()  # Rollback der Transaktion im Fehlerfall

finally:
    # Cursor und Verbindung schließen
    if cur:
        cur.close()
    if conn:
        conn.close()


In [31]:
# Schritt 6 (Teil 2 von 2): Erweiterte Daten in Tabelle "spieler" importieren

# Pfad zur JSON-Datei
json_file_path = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/footballmap_v0.1/alle_spieler_daten.json"

# Funktion zum Umwandeln von Datumsangaben in SQL-Datumsformat
def convert_date(date_str):
    months = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "May": "05",
        "Jun": "06",
        "Jul": "07",
        "Aug": "08",
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
        "Dec": "12"
    }
    
    parts = date_str.split()
    
    if len(parts) != 3:
        return None
    
    month = months.get(parts[0], '01')
    day = parts[1].strip(',')
    year = parts[2]
    
    return f"{year}-{month}-{day}"

try:
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for player_data in data:
        spieler_nr = player_data.get('spieler_nr', None)
        
        spielerbild_link = player_data.get('spielerbild_link', '')[:255] if player_data.get('spielerbild_link') else None
        geburtsort = player_data.get('geburtsort', 'Unknown')[:100]
        geburtsland = player_data.get('geburtsland', 'Unknown')[:100]
        
        # Überprüfung für die Trikotnummer
        trikotnummer = player_data.get('trikotnummer', None)
        if trikotnummer and trikotnummer != 'Nicht gefunden':
            trikotnummer = int(trikotnummer)
        else:
            trikotnummer = None
        
        vertragsende = convert_date(player_data.get('vertragsende', 'Jan 1, 1970'))

        update_query = """
        UPDATE Spieler 
        SET spielerbild_link = %s, geburtsort = %s, geburtsland = %s, trikotnummer = %s, vertragsende = %s
        WHERE spieler_nr = %s;
        """
        data_tuple = (spielerbild_link, geburtsort, geburtsland, trikotnummer, vertragsende, spieler_nr)
        cur.execute(update_query, data_tuple)

    conn.commit()

except Exception as e:
    print(f"Fehler beim Aktualisieren der Daten: {e}")
    conn.rollback()


In [44]:
# Marktwerte korrigieren (SQL-Code direkt in der Datenbank ausführen)

UPDATE Spieler
SET marktwert_aktuell = marktwert_aktuell * 1000000
WHERE marktwert_aktuell < 100;



In [ ]:
# Schritt 7: Import Transferdaten

import csv
import psycopg2
from psycopg2 import sql
import logging
import re

# Logging konfigurieren
logging.basicConfig(level=logging.INFO)

try:
    conn = psycopg2.connect(
        dbname="footballmap",
        user="postgres",
        password="xxx",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    csv_file = "C:\\Users\\ssidl\\Documents\\_Projekte_FHNW_Lokal\\Semester_6\\GeoInformatik\\old\\footballmap_v0.1\\CH_SuperLeague_allClubs_transferdata.csv"

    with open(csv_file, mode='r', encoding='utf-8-sig') as file:
        reader = csv.DictReader(file, delimiter=';')
        
        for row in reader:
            try:
                spieler_nr = int(row['Spieler Nr'])

                # Überprüfen, ob der Spieler bereits existiert
                cur.execute("SELECT spieler_nr FROM spieler WHERE spieler_nr = %s", (spieler_nr,))
                if cur.fetchone() is None:
                    logging.warning(f"Der Spieler mit der Nummer {spieler_nr} existiert nicht in der Datenbank. Zeile wird übersprungen.")
                    continue

                default_club = 999998

                von_club_nr = int(row['old club nr']) if row['old club nr'] else default_club
                cur.execute("SELECT club_nr FROM Club WHERE club_nr = %s", (von_club_nr,))
                if cur.fetchone() is None:
                    von_club_nr = 999999

                nach_club_nr = int(row['new club nr']) if row['new club nr'] else default_club
                cur.execute("SELECT club_nr FROM Club WHERE club_nr = %s", (nach_club_nr,))
                if cur.fetchone() is None:
                    nach_club_nr = 999999

                transfer_sum = row['transfer sum']
                marktwert_str = row['market value']

                transfer_sum_num = re.findall(r'\d+\.?\d*', transfer_sum)
                marktwert_num = re.findall(r'\d+\.?\d*', marktwert_str)

                multiplier = 1
                if 'Mio' in transfer_sum:
                    multiplier = 1000000
                elif 'Tsd' in transfer_sum:
                    multiplier = 1000

                ablösesumme = float(transfer_sum_num[0]) * multiplier if transfer_sum_num else 0
                marktwert = float(marktwert_num[0]) * 1000000 if 'Mio' in marktwert_str else float(marktwert_num[0]) * 1000 if 'Tsd' in marktwert_str else 0

                datum = row['date']
                von_land = row['old club country']
                nach_land = row['new club country']

                transferart = transfer_sum if transfer_sum in ['Leihe', 'Leih-Ende'] else None

                insert_query = sql.SQL("""
                    INSERT INTO Transfer (von_club, nach_club, spieler_nr, ablösesumme, marktwert, datum, transferart, von_land, nach_land)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """)

                cur.execute(insert_query, (
                    von_club_nr, nach_club_nr, spieler_nr, ablösesumme, marktwert, datum, transferart, von_land, nach_land
                ))

            except Exception as e:
                logging.error(f"Fehler beim Verarbeiten der Zeile: {row}. Fehler: {e}")

        conn.commit()
        logging.info("Daten wurden erfolgreich in die Datenbank eingefügt!")

except Exception as e:
    logging.error(f"Fehler bei der Datenbankverbindung oder während des Imports: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()


In [ ]:
# leere einträge bei Transferart füllen

UPDATE Transfer
SET TransferArt = 'Transfer'
WHERE TransferArt IS NULL;


In [ ]:
# Schritt 8

# Die Club-Logos basierend auf der Spalte "logo_link" in der Tabelle "club" lokal auf dem PC abspeichern
# bereits bestehende Logos werden übersprungen

import os

# Verzeichnis zum Speichern der Logos festlegen
output_directory = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/png/Logos"

# SQL-Abfrage vorbereiten, um logo_link und club_nr abzurufen
select_query = """
    SELECT club_nr, logo_link
    FROM Club
"""

try:
    # Abfrage ausführen
    cur.execute(select_query)
    
    # Ergebnisse abrufen
    rows = cur.fetchall()
    
    # Für jedes Logo den Link abrufen und lokal speichern, wenn es noch nicht vorhanden ist
    for row in rows:
        club_nr = row[0]
        logo_link = row[1]
        
        # Dateiname aus dem Link extrahieren
        filename = f"{club_nr}.png"
        filepath = os.path.join(output_directory, filename)
        
        # Prüfen, ob das Logo bereits vorhanden ist
        if not os.path.exists(filepath):
            # Logo herunterladen
            response = requests.get(logo_link)
            
            if response.status_code == 200:
                # Logo lokal speichern
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"Logo für Verein mit der Nummer {club_nr} wurde als {filename} gespeichert.")
            else:
                print(f"Fehler beim Herunterladen des Logos für Verein mit der Nummer {club_nr}: Statuscode {response.status_code}")
        else:
            print(f"Logo für Verein mit der Nummer {club_nr} ist bereits vorhanden.")
    
except Exception as e:
    print("Fehler beim Abrufen und Speichern der Logos:", e)


In [ ]:
# Schritt 9

# Spalten "flagge" (Blob) und "flagge_link" in der Tabelle "club" abfüllen. 
# Dazu werden die zuvor (Schritt 8) lokal abgespeicherten .png-Dateien verwendet

# Verzeichnis, in dem die Logos gespeichert sind
logos_directory = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/png/Logos"

# SQL-Abfrage vorbereiten, um club_nr und logo_pfad abzurufen
select_query = """
    SELECT club_nr, logo_pfad
    FROM Club
"""

try:
    # Abfrage ausführen
    cur.execute(select_query)
    
    # Ergebnisse abrufen
    rows = cur.fetchall()
    
    # Für jedes Logo den Club in der Datenbank aktualisieren
    for row in rows:
        club_nr = row[0]
        logo_pfad = row[1]
        
        # Dateipfad für das Logo erstellen (mit Schrägstrichen)
        logo_filepath = os.path.join(logos_directory, f"{club_nr}.png").replace("\\", "/")
        
        # Logo als Blob einlesen und in die Datenbank einfügen, falls das Logo vorhanden ist
        if os.path.exists(logo_filepath):
            with open(logo_filepath, 'rb') as f:
                logo_data = f.read()
                cur.execute("UPDATE Club SET logo = %s WHERE club_nr = %s", (psycopg2.Binary(logo_data), club_nr))
            
            # Logo-Pfad in der Datenbank aktualisieren
            cur.execute("UPDATE Club SET logo_pfad = %s WHERE club_nr = %s", (logo_filepath, club_nr))
            
            print(f"Logo und Logo-Pfad für Verein mit der Nummer {club_nr} wurden erfolgreich aktualisiert.")
        else:
            print(f"Kein Logo gefunden für Verein mit der Nummer {club_nr}. Überspringe...")
    
    # Transaktion bestätigen
    conn.commit()
    
    print("Alle Logos und Logo-Pfade wurden erfolgreich aktualisiert.")
    
except Exception as e:
    # Transaktion rückgängig machen und Fehler ausgeben
    conn.rollback()
    print("Fehler beim Aktualisieren der Logos und Logo-Pfade:", e)


In [ ]:
# falls der lokale Pfad ändert, kann folgender SQL-Befehl auf der DB ausgeführt werden:
# Pfad anpassen!

UPDATE Land
SET flagge_pfad = replace('C:\Users\ssidl\Documents\_Projekte_FHNW_Lokal\Semester_6\GeoInformatik\png\Flaggen\' || ländercode || '.png', '\', '/');

In [ ]:
# Schritt 10

# Die Spieler-Bilder basierend auf der Spalte "spielerbild_link" in der Tabelle "spieler" lokal auf dem PC abspeichern
# bereits bestehende Bilder werden übersprungen

import os

# Verzeichnis zum Speichern der Bilder festlegen
output_directory = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/png/Spieler"

# SQL-Abfrage vorbereiten, um spielerbild_link und spieler_nr abzurufen
select_query = """
    SELECT spieler_nr, spielerbild_link
    FROM Spieler
"""

try:
    # Abfrage ausführen
    cur.execute(select_query)
    
    # Ergebnisse abrufen
    rows = cur.fetchall()
    
    # Für jedes Bild den Link abrufen und lokal speichern, wenn es noch nicht vorhanden ist
    for row in rows:
        spieler_nr = row[0]
        bild_link = row[1]
        
        # Dateiname aus dem Link extrahieren
        filename = f"{spieler_nr}.png"
        filepath = os.path.join(output_directory, filename)
        
        # Prüfen, ob das Bild bereits vorhanden ist
        if not os.path.exists(filepath):
            # Bild herunterladen
            response = requests.get(bild_link)
            
            if response.status_code == 200:
                # Bild lokal speichern
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"Bild für Spieler mit der Nummer {spieler_nr} wurde als {filename} gespeichert.")
            else:
                print(f"Fehler beim Herunterladen des Bilds für Spieler mit der Nummer {spieler_nr}: Statuscode {response.status_code}")
        else:
            print(f"Bild für Spieler mit der Nummer {spieler_nr} ist bereits vorhanden.")
    
except Exception as e:
    print("Fehler beim Abrufen und Speichern der Bilder:", e)


In [ ]:
# Schritt 11

# Spalten "spielerbild" (Blob) und "spielerbild_pfad" in der Tabelle "spieler" abfüllen. 
# Dazu werden die zuvor (Schritt 10) lokal abgespeicherten .png-Dateien verwendet

# Verzeichnis, in dem die Spielerbilder gespeichert sind
spielerbilder_directory = "C:/Users/ssidl/Documents/_Projekte_FHNW_Lokal/Semester_6/GeoInformatik/old/png/Spieler"

# SQL-Abfrage vorbereiten, um spieler_nr und spielerbild_pfad abzurufen
select_query = """
    SELECT spieler_nr, spielerbild_pfad
    FROM Spieler
"""

try:
    # Abfrage ausführen
    cur.execute(select_query)
    
    # Ergebnisse abrufen
    rows = cur.fetchall()
    
    # Für jedes Spielerbild den Spieler in der Datenbank aktualisieren
    for row in rows:
        spieler_nr = row[0]
        spielerbild_pfad = row[1]
        
        # Dateipfad für das Spielerbild erstellen (mit Schrägstrichen)
        spielerbild_filepath = os.path.join(spielerbilder_directory, f"{spieler_nr}.png").replace("\\", "/")
        
        # Spielerbild als Blob einlesen und in die Datenbank einfügen, falls das Bild vorhanden ist
        if os.path.exists(spielerbild_filepath):
            with open(spielerbild_filepath, 'rb') as f:
                spielerbild_data = f.read()
                cur.execute("UPDATE Spieler SET spielerbild = %s WHERE spieler_nr = %s", (psycopg2.Binary(spielerbild_data), spieler_nr))
            
            # Spielerbild-Pfad in der Datenbank aktualisieren
            cur.execute("UPDATE Spieler SET spielerbild_pfad = %s WHERE spieler_nr = %s", (spielerbild_filepath, spieler_nr))
            
            print(f"Spielerbild und Spielerbild-Pfad für Spieler mit der Nummer {spieler_nr} wurden erfolgreich aktualisiert.")
        else:
            print(f"Kein Spielerbild gefunden für Spieler mit der Nummer {spieler_nr}. Überspringe...")
    
    # Transaktion bestätigen
    conn.commit()
    
    print("Alle Spielerbilder und Spielerbild-Pfade wurden erfolgreich aktualisiert.")
    
except Exception as e:
    # Transaktion rückgängig machen und Fehler ausgeben
    conn.rollback()
    print("Fehler beim Aktualisieren der Spielerbilder und Spielerbild-Pfade:", e)


In [ ]:
# falls der lokale Pfad ändert, kann folgender SQL-Befehl auf der DB ausgeführt werden:
# Pfad anpassen!

UPDATE spieler
SET spielerbild_pfad = replace('C:\Users\ssidl\Documents\_Projekte_FHNW_Lokal\Semester_6\GeoInformatik\png\Spieler\' || spieler_nr || '.png', '\', '/');

In [41]:
# Cursor und Verbindung schließen
cur.close()
conn.close()